In [14]:
import os
import cv2
import numpy as np
from PIL import Image

In [3]:
def read_image(image_path):
    #Read image using PIL and convert to NumPy array
    image = Image.open(image_path)
    return np.array(image)

def resize_image(image, size=(512, 512)):
    #Resize image to a uniform size
    return cv2.resize(image, size)

def to_grayscale(image):
    #Convert RGB image to grayscale
    if len(image.shape) == 3:
        return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return image

def apply_clahe(image):
    #Apply CLAHE for contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

def apply_roi_mask(image, roi_mask):
    #Apply circular ROI mask to focus on the retina region only
    return cv2.bitwise_and(image, image, mask=roi_mask)

def normalize_image(image):
    #Normalize pixel values to [0, 1]
    return image.astype(np.float32) / 255.0

def binarize_mask(mask):
    #Convert grayscale mask to binary mask
    return (mask > 0).astype(np.uint8)

def preprocess_drive_image(image_path, roi_mask_path=None, size=(512, 512)):
    image = read_image(image_path)
    image = resize_image(image, size)
    image = to_grayscale(image)
    image = apply_clahe(image)

    if roi_mask_path:
        roi_mask = read_image(roi_mask_path)
        roi_mask = resize_image(roi_mask, size)
        image = apply_roi_mask(image, roi_mask)

    image = normalize_image(image)
    return image  
def preprocess_drive_mask(mask_path, size=(512, 512)):
    mask = read_image(mask_path)
    mask = resize_image(mask, size)
    mask = binarize_mask(mask)
    return mask  # shape: (H, W) or (1, H, W) for DL input

hub://activeloop/drive-train loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/drive-train
hub://activeloop/drive-test loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/drive-test


In [13]:
print(train_ds.tensors)
print(test_ds.tensors)


{'rgb_images': Tensor(key='rgb_images'), 'manual_masks/mask': Tensor(key='manual_masks/mask'), 'masks/mask': Tensor(key='masks/mask')}
{'rgb_images': Tensor(key='rgb_images'), 'masks': Tensor(key='masks')}


In [12]:
def show_sample(dataset, index=0, title="Sample"):
    img = dataset['rgb_images'][index].numpy()
    mask = dataset['manual_masks/mask'][index].numpy()

    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.title(f"{title} Image")
    plt.imshow(img)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title(f"{title} Mask")
    plt.imshow(mask, cmap='gray')
    plt.axis('off')

    plt.tight_layout()
    plt.show()



In [ ]:
# Read all images path
image_paths = get_image_paths(TEST_IMG_BASE_PATHS)

# Read image using PIL and convert to numpy
test_images = []
for path in image_paths:
    image = Image.open(path)
    image = np.array(image)
    test_images.append(image)
    
# Plot images
plot_image_grid(test_images)

In [ ]:
# Assuming test_images is a list of NumPy arrays (grayscale images)
segmented_images = []

for image in test_images:
    # Step 1: Convert to grayscale (if the image is not already grayscale)
    if len(image.shape) == 3:  # Check if the image has multiple channels (RGB)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 2: Preprocessing - Contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(image)
    # Step 3: Morphological operations (e.g., opening)
    selem = disk(6)  # Adjust disk size based on vessel scale
    blackhat_image = black_tophat(enhanced_image, selem)
    
    # Step 4: Thresholding to segment vessels
    # Otsu's thresholding
    _, binary_image = cv2.threshold(blackhat_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Step5: Store the segmented image
    segmented_images.append(binary_image)

In [ ]:
# Plot images
plot_image_grid(segmented_images)